In [1]:
import json
import boto3
from sagemaker.local import LocalSession
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

DUMMY_IAM_ROLE = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

boto3_session = boto3.Session(region_name="us-east-1")
session = LocalSession(boto3_session)
session.config = {"local": {"local_code": True}}

In [21]:
role = DUMMY_IAM_ROLE
model_dir = "file://ml/model"

model = Model(
    predictor_cls=Predictor,
    image_uri="zeroae/sagemaker-darknet-inference",
    model_data=model_dir,
    role=DUMMY_IAM_ROLE,
    sagemaker_session=session,
)

In [22]:
predictor = model.deploy(
    instance_type="local",
    initial_instance_count=1,
    serializer=IdentitySerializer("image/jpeg"),
    deserializer=JSONDeserializer(),
)

Attaching to 8y2hfvaxhd-algo-1-9j1rc
8y2hfvaxhd-algo-1-9j1rc | Warning: MMS is using non-default JVM parameters: -XX:-UseContainerSupport
8y2hfvaxhd-algo-1-9j1rc | 2021-11-03 21:19:22,257 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
8y2hfvaxhd-algo-1-9j1rc | MMS Home: /opt/conda/envs/darknet/lib/python3.8/site-packages
8y2hfvaxhd-algo-1-9j1rc | Current directory: /
8y2hfvaxhd-algo-1-9j1rc | Temp directory: /tmp
8y2hfvaxhd-algo-1-9j1rc | Number of GPUs: 0
8y2hfvaxhd-algo-1-9j1rc | Number of CPUs: 6
8y2hfvaxhd-algo-1-9j1rc | Max heap size: 1486 M
8y2hfvaxhd-algo-1-9j1rc | Python executable: /opt/conda/envs/darknet/bin/python3.8
8y2hfvaxhd-algo-1-9j1rc | Config file: /etc/sagemaker-mms.properties
8y2hfvaxhd-algo-1-9j1rc | Inference address: http://0.0.0.0:8080
8y2hfvaxhd-algo-1-9j1rc | Management address: http://0.0.0.0:8080
8y2hfvaxhd-algo-1-9j1rc | Model Store: /.sagemaker/mms/models
8y2hfvaxhd-algo-1-9j1rc | Initial Models: ALL
8y2hfvaxhd-algo-1-9j1rc | Log dir: /logs
8y2hfvaxhd-al

In [23]:
with open("dog.jpg", "rb") as f:
    predictions = predictor.predict(
        f.read()
    )

8y2hfvaxhd-algo-1-9j1rc | 2021-11-03 21:20:48,827 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 23483
8y2hfvaxhd-algo-1-9j1rc | 2021-11-03 21:20:48,831 [INFO ] W-9000-model ACCESS_LOG - /172.19.0.1:58950 "POST /invocations HTTP/1.1" 200 23496


In [24]:
print(json.dumps(predictions, indent=2))

{
  "Labels": [
    {
      "Name": "bicycle",
      "Confidence": 92.4040675163269,
      "Instances": [
        {
          "Confidence": 92.4040675163269,
          "BoundingBox": {
            "Width": 458.94085693359375,
            "Height": 273.18804931640625,
            "Left": 114.1905517578125,
            "Top": 410.19049072265625
          }
        }
      ],
      "Parents": []
    },
    {
      "Name": "truck",
      "Confidence": 86.79984211921692,
      "Instances": [
        {
          "Confidence": 86.79984211921692,
          "BoundingBox": {
            "Width": 227.93612670898438,
            "Height": 93.27326202392578,
            "Left": 467.2970733642578,
            "Top": 169.18019485473633
          }
        }
      ],
      "Parents": []
    },
    {
      "Name": "dog",
      "Confidence": 96.15262150764465,
      "Instances": [
        {
          "Confidence": 96.15262150764465,
          "BoundingBox": {
            "Width": 181.7491912841797,
    

In [19]:
predictor.delete_predictor()

Gracefully stopping... (press Ctrl+C again to force)
